### Сбор цен на книги

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [11]:
url = 'http://books.toscrape.com/catalogue/page-1.html'

response = requests.get(url)
response

<Response [200]>

##### HTML-разметка ответа в виде дерева

In [13]:
tree = BeautifulSoup(response.content, 'html.parser')
tree


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:30" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link 

In [14]:
type(tree)

bs4.BeautifulSoup

##### Заголовок HTML-страницы

In [18]:
tree.html.head.title.text.strip()

'All products | Books to Scrape - Sandbox'

##### Поиск объекта по его адресу

In [20]:
tree.find('article', {'class': 'product_pod'})

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

##### Количество всех найденных тегов

In [21]:
len(tree.find_all('article', {'class': 'product_pod'}))

20

##### Первый элемент запроса

In [22]:
books = tree.find_all('article', {'class' : 'product_pod'})

# Первый элемент
books[0]

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

##### Поиск вложенного тега

In [23]:
books[0].h3

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

In [24]:
books[0].h3.a

<a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

##### Текстовое содержимое по ссылке

In [26]:
books[0].h3.a.text

'A Light in the ...'

**Метод get('title') возвращает значение атрибута по ссылке**

In [27]:
books[0].h3.a.get('title')

'A Light in the Attic'

##### Названия книг

In [28]:
for i in range(len(books)):
    print(books[i].h3.a.get('title'))

A Light in the Attic
Tipping the Velvet
Soumission
Sharp Objects
Sapiens: A Brief History of Humankind
The Requiem Red
The Dirty Little Secrets of Getting Your Dream Job
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
The Black Maria
Starving Hearts (Triangular Trade Trilogy, #1)
Shakespeare's Sonnets
Set Me Free
Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Rip it Up and Start Again
Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Olio
Mesaerion: The Best Science Fiction Stories 1800-1849
Libertarianism for Beginners
It's Only the Himalayas


##### Рейтинг книг

In [29]:
books[0].p.get('class')

['star-rating', 'Three']

In [30]:
books[0].p.get('class')[1]

'Three'

##### Цены книг

In [31]:
books[0].find('p', {'class': 'price_color'})

<p class="price_color">£51.77</p>

In [32]:
books[0].find('p', {'class': 'price_color'}).text

'£51.77'

### Создание датафрейма

In [33]:
books = tree.find_all('article', {'class' : 'product_pod'})

In [34]:
total_book = []

for book in books:
    title = book.h3.a.get('title')
    rating = book.p.get('class')[1]
    price = float(book.find('p', {'class': 'price_color'}).text[1:])
    
    book_dict = {
        'title': title,
        'rating': rating,
        'price': price
    }
    
    total_book.append(book_dict)

In [35]:
print(total_book[:3])

[{'title': 'A Light in the Attic', 'rating': 'Three', 'price': 51.77}, {'title': 'Tipping the Velvet', 'rating': 'One', 'price': 53.74}, {'title': 'Soumission', 'rating': 'One', 'price': 50.1}]


##### Датафрейм

In [37]:
df = pd.DataFrame(total_book)
df.head(5)

,title,rating,price
0,A Light in the Attic,Three,51.77
1,Tipping the Velvet,One,53.74
2,Soumission,One,50.10
3,Sharp Objects,Four,47.82
4,Sapiens: A Brief History of Humankind,Five,54.23


##### Перебираем все страницы

In [38]:
def get_page(p):
    # Подготовка ссылки
    url = f'http://books.toscrape.com/catalogue/page-{p}.html'

    # Прошли по ссылке
    response = requests.get(url)

    # Построение дерева
    tree = BeautifulSoup(response.content, 'html.parser')

    # Поиск книг
    books = tree.find_all('article', {'class' : 'product_pod'})

    info = []
    for book in books:
        title = book.h3.a.get('title')
        ranking = book.p.get('class')[1]
        price = book.find('p', {'class': 'price_color'}).text
        url = 'http://books.toscrape.com/catalogue/' + book.div.a.get('href')

        row = {
            'title': title,
            'price': price,
            'ranking': ranking,
            'url': url
            }
        info.append(row)

    return info

##### Запускаем цикл

In [40]:
infa = []

for p in range(1, 51):
    infa.extend(get_page(p))

In [41]:
len(infa)

1000

##### Преобразуем в датафрейм

In [42]:
df = pd.DataFrame(infa)
df.head(5)

,title,price,ranking,url
0,A Light in the Attic,£51.77,Three,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,£53.74,One,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,£50.10,One,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,£47.82,Four,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,£54.23,Five,http://books.toscrape.com/catalogue/sapiens-a-...


In [43]:
df.shape

(1000, 4)